In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Paraná - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.284196,0.300568,332.705560,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,189.804
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.277973,0.301710,332.945945,2.029477e+08,1.061597e+07,17.592548,1.852326e+08,0.776975,174.713
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.271750,0.302645,335.222130,2.031806e+08,1.062582e+07,17.595559,1.852864e+08,0.777079,180.801
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.265527,0.303413,334.785737,2.034136e+08,1.063567e+07,17.598570,1.853401e+08,0.777183,180.172
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.259304,0.304034,335.050185,2.036466e+08,1.064551e+07,17.601581,1.853939e+08,0.777287,185.597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549782,616.073841,NaN,NaN,NaN,NaN,NaN,412.177
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546864,614.423079,NaN,NaN,NaN,NaN,NaN,337.056
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544541,613.772150,NaN,NaN,NaN,NaN,NaN,340.018
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540629,614.761140,NaN,NaN,NaN,NaN,NaN,383.477


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Paraná - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.130907,0.732054,0.692024,1.202413,-0.413954,0.937022,1.076355,1.563435
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.076074,0.695547,0.722303,1.204670,-0.429516,0.934746,1.074731,1.542097
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.021241,0.667742,0.729289,1.206927,-0.445078,0.932469,1.073107,1.520760
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.033592,0.634897,0.709776,1.209183,-0.460640,0.930193,1.071484,1.499423


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
157    280.736
158    352.332
159    280.856
160    319.397
161    295.932
Name: Paraná - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,-1.195259,1.373912,0.887005,0.826002,0.454561,0.988596,0.979507,0.670392
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,-1.109331,1.418215,0.875394,0.842968,0.424602,0.981838,0.982236,0.678142
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,-1.023403,1.454067,0.855534,0.859935,0.394643,0.975081,0.984964,0.685892
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,-0.937476,1.490026,0.842489,0.876901,0.364683,0.968323,0.987693,0.693642
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,-0.851548,1.513267,0.820774,0.893867,0.334724,0.961566,0.990422,0.701392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[528538996, 218594261, 1739742744, 2988705785, 672415278, 726566307, 3891586175, 531457047, 3238361904, 497112064, 569167671, 3611468519, 4187090081, 3732494524, 3261764637, 3407663339, 3125135825, 994007427, 1546219807, 1323155378, 1407932762, 602823176, 3953758461, 1096676460, 1735749876, 888241683, 1032407587, 3405257239, 240115176, 4137093699, 2764324150, 832988002, 1110872390, 1255647697, 3742403372, 4170544142, 301743673, 735184766, 3827189393, 1529396637, 7422971, 1871095922, 294809271, 1687682974, 2144459914, 1747776257, 2730196796, 3172964387, 4001146587, 41891827]


Step: 0 ___________________________________________
val_loss: 436.566162109375
winner_seed: 528538996


Step: 1 ___________________________________________
val_loss: 324.3874816894531
winner_seed: 218594261


Step: 2 ___________________________________________
val_loss: 359.4514465332031


Step: 3 ___________________________________________
val_loss: 382.9853515625


Step: 4 _______________________________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 135ms/step - loss: 73633.7109 - val_loss: 4448.4619
Epoch 2/10000
4/4 [==============================] - 0s 13ms/step - loss: 5069.2251 - val_loss: 3787.6233
Epoch 3/10000
4/4 [==============================] - 0s 12ms/step - loss: 4781.5571 - val_loss: 3337.4707
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 4236.3965 - val_loss: 3253.0576
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 4367.7603 - val_loss: 3439.6050
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 4245.3975 - val_loss: 3668.6433
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 4221.2939 - val_loss: 3071.5225
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 4042.6721 - val_loss: 4774.7354
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 4199.9844 - val_loss: 2924.6311
Epoch 10/10000
4/4 [========================

4/4 [==============================] - 0s 12ms/step - loss: 541.9348 - val_loss: 612.8893
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 853.8602 - val_loss: 437.4491
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 691.0261 - val_loss: 650.4494
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 692.0317 - val_loss: 447.1636
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 646.8665 - val_loss: 533.2261
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 568.6753 - val_loss: 621.3977
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 664.0797 - val_loss: 1084.5950
Epoch 85/10000
4/4 [==============================] - 0s 13ms/step - loss: 1105.6217 - val_loss: 446.3141
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 893.2454 - val_loss: 578.0104
Epoch 87/10000
4/4 [==============================] - 0s 14ms/step -

4/4 [==============================] - 0s 11ms/step - loss: 627.0157 - val_loss: 684.9048
Epoch 156/10000
4/4 [==============================] - 0s 11ms/step - loss: 631.9897 - val_loss: 720.5947
Epoch 157/10000
4/4 [==============================] - 0s 11ms/step - loss: 598.4394 - val_loss: 475.5174
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 545.8951 - val_loss: 993.8140
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 772.5193 - val_loss: 551.4898
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 560.9909 - val_loss: 634.4731
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 565.5708 - val_loss: 665.6392
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 578.5229 - val_loss: 535.6050
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 536.8497 - val_loss: 564.8234
Epoch 164/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 12ms/step - loss: 460.2156 - val_loss: 993.5727
Epoch 233/10000
4/4 [==============================] - 0s 12ms/step - loss: 634.1497 - val_loss: 908.4050
Epoch 234/10000
4/4 [==============================] - 0s 12ms/step - loss: 688.0204 - val_loss: 495.7777
Epoch 235/10000
4/4 [==============================] - 0s 12ms/step - loss: 440.6081 - val_loss: 733.2952
Epoch 236/10000
4/4 [==============================] - 0s 14ms/step - loss: 459.7641 - val_loss: 598.1520
Epoch 237/10000
4/4 [==============================] - 0s 12ms/step - loss: 478.7984 - val_loss: 552.9728
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 481.0318 - val_loss: 704.9669
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 617.9870 - val_loss: 565.4374
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 436.9440 - val_loss: 484.9297
Epoch 241/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 11ms/step - loss: 396.0254 - val_loss: 717.1512
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 468.6530 - val_loss: 570.3399
Epoch 311/10000
4/4 [==============================] - 0s 12ms/step - loss: 402.7355 - val_loss: 758.2689
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 492.0177 - val_loss: 590.3951
Epoch 313/10000
4/4 [==============================] - 0s 11ms/step - loss: 410.4725 - val_loss: 617.4619
Epoch 314/10000
4/4 [==============================] - 0s 12ms/step - loss: 467.6088 - val_loss: 645.6909
Epoch 315/10000
4/4 [==============================] - 0s 10ms/step - loss: 431.7957 - val_loss: 599.2899
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 407.7705 - val_loss: 657.2218
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 490.5135 - val_loss: 640.7917
Epoch 318/10000
4/4 [==============================] - 0s 11ms

...

4/4 [==============================] - 0s 11ms/step - loss: 149.4837 - val_loss: 426.1425
Epoch 1994/10000
4/4 [==============================] - 0s 11ms/step - loss: 176.6928 - val_loss: 393.0713
Epoch 1995/10000
4/4 [==============================] - 0s 11ms/step - loss: 125.8658 - val_loss: 398.1542
Epoch 1996/10000
4/4 [==============================] - 0s 11ms/step - loss: 115.5604 - val_loss: 331.9509
Epoch 1997/10000
4/4 [==============================] - 0s 10ms/step - loss: 136.2480 - val_loss: 383.4125
Epoch 1998/10000
4/4 [==============================] - 0s 11ms/step - loss: 146.9211 - val_loss: 432.0343
Epoch 1999/10000
4/4 [==============================] - 0s 10ms/step - loss: 143.5544 - val_loss: 351.3417
Epoch 2000/10000
4/4 [==============================] - 0s 11ms/step - loss: 131.2655 - val_loss: 381.1686
Epoch 2001/10000
4/4 [==============================] - 0s 11ms/step - loss: 138.0824 - val_loss: 430.0539
Epoch 2002/10000
4/4 [==============================] 

Epoch 2070/10000
4/4 [==============================] - 0s 11ms/step - loss: 124.9234 - val_loss: 273.5123
Epoch 2071/10000
4/4 [==============================] - 0s 10ms/step - loss: 169.9955 - val_loss: 288.2999
Epoch 2072/10000
4/4 [==============================] - 0s 11ms/step - loss: 155.2632 - val_loss: 282.6464
Epoch 2073/10000
4/4 [==============================] - 0s 11ms/step - loss: 118.4688 - val_loss: 421.9143
Epoch 2074/10000
4/4 [==============================] - 0s 13ms/step - loss: 123.2593 - val_loss: 375.5683
Epoch 2075/10000
4/4 [==============================] - 0s 12ms/step - loss: 180.9271 - val_loss: 430.5149
Epoch 2076/10000
4/4 [==============================] - 0s 13ms/step - loss: 142.9463 - val_loss: 447.0211
Epoch 2077/10000
4/4 [==============================] - 0s 11ms/step - loss: 165.1333 - val_loss: 440.2200
Epoch 2078/10000
4/4 [==============================] - 0s 12ms/step - loss: 113.5300 - val_loss: 419.1490
Epoch 2079/10000
4/4 [===============

4/4 [==============================] - 0s 9ms/step - loss: 135.0577 - val_loss: 447.3395
Epoch 2147/10000
4/4 [==============================] - 0s 10ms/step - loss: 114.5672 - val_loss: 441.7911
Epoch 2148/10000
4/4 [==============================] - 0s 10ms/step - loss: 123.9893 - val_loss: 508.1508
Epoch 2149/10000
4/4 [==============================] - 0s 11ms/step - loss: 114.7887 - val_loss: 518.8314
Epoch 2150/10000
4/4 [==============================] - 0s 10ms/step - loss: 133.1098 - val_loss: 397.8437
Epoch 2151/10000
4/4 [==============================] - 0s 11ms/step - loss: 122.5452 - val_loss: 395.6317
Epoch 2152/10000
4/4 [==============================] - 0s 10ms/step - loss: 108.6071 - val_loss: 344.5754
Epoch 2153/10000
4/4 [==============================] - 0s 10ms/step - loss: 143.9430 - val_loss: 491.9516
Epoch 2154/10000
4/4 [==============================] - 0s 10ms/step - loss: 110.3446 - val_loss: 465.2070
Epoch 2155/10000
4/4 [==============================] -

Epoch 2223/10000
4/4 [==============================] - 0s 13ms/step - loss: 189.0816 - val_loss: 393.4657
Epoch 2224/10000
4/4 [==============================] - 0s 12ms/step - loss: 192.0036 - val_loss: 334.1835
Epoch 2225/10000
4/4 [==============================] - 0s 12ms/step - loss: 153.0464 - val_loss: 346.5029
Epoch 2226/10000
4/4 [==============================] - 0s 12ms/step - loss: 173.6920 - val_loss: 463.7956
Epoch 2227/10000
4/4 [==============================] - 0s 10ms/step - loss: 194.3489 - val_loss: 463.3454
Epoch 2228/10000
4/4 [==============================] - 0s 12ms/step - loss: 158.7867 - val_loss: 459.1505
Epoch 2229/10000
4/4 [==============================] - 0s 11ms/step - loss: 140.5070 - val_loss: 395.0962
Epoch 2230/10000
4/4 [==============================] - 0s 11ms/step - loss: 172.9215 - val_loss: 411.8331
Epoch 2231/10000
4/4 [==============================] - 0s 11ms/step - loss: 184.1113 - val_loss: 367.5756
Epoch 2232/10000
4/4 [===============

4/4 [==============================] - 0s 11ms/step - loss: 84.1100 - val_loss: 507.5843
Epoch 2300/10000
4/4 [==============================] - 0s 13ms/step - loss: 93.0068 - val_loss: 445.7492
Epoch 2301/10000
4/4 [==============================] - 0s 11ms/step - loss: 100.5976 - val_loss: 335.9211
Epoch 2302/10000
4/4 [==============================] - 0s 11ms/step - loss: 87.1208 - val_loss: 383.7555
Epoch 2303/10000
4/4 [==============================] - 0s 12ms/step - loss: 98.6582 - val_loss: 526.4009
Epoch 2304/10000
4/4 [==============================] - 0s 12ms/step - loss: 108.8118 - val_loss: 357.3421
Epoch 2305/10000
4/4 [==============================] - 0s 11ms/step - loss: 102.4027 - val_loss: 433.8823
Epoch 2306/10000
4/4 [==============================] - 0s 11ms/step - loss: 120.9577 - val_loss: 465.2110
Epoch 2307/10000
4/4 [==============================] - 0s 11ms/step - loss: 122.6450 - val_loss: 517.3006
Epoch 2308/10000
4/4 [==============================] - 0s

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 19ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,292.434479,315.98175,289.119843,308.613953,307.056824,293.898499,313.536407,308.02594,309.053162,309.679047,309.667969,309.560913,305.749634,309.084961,308.084412,308.141083,308.236542,306.281433,308.369751,306.816376,307.535187,306.204956,305.622009,304.492737,306.315735,304.572754,306.207642,306.213135,310.021027,306.696167
Target,354.558,360.106,350.123,288.254,301.687,268.901,274.431,298.219,328.955,335.039,260.723,366.929,353.426,344.937,302.082,294.467,336.029,278.117,323.005,291.453,300.203,317.058,314.142,301.878,358.797,356.169,316.188,346.214,323.401,238.466
Error,62.123535,44.124237,61.003143,20.359955,5.369812,24.997498,39.105408,9.806946,19.901825,25.359955,48.944977,57.368073,47.676361,35.852051,6.002411,13.674072,27.79245,28.164429,14.635254,15.363373,7.332184,10.853058,8.519989,2.614746,52.481262,51.596252,9.980347,40.000854,13.379974,68.230164


In [38]:
display(mae)
display(mape)

29.087152

0.0918777

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined